In [5]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('Assets\CONVICTIONSUMMARY_270_opt3_best2(1).csv', sep='|')

In [16]:
df2 = pd.DataFrame()
df2 = df.loc[df['TYPE'] == 'MERGED']

Extract Time and Date

In [ ]:
df2["Time"] = df2.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)

df2["DATE"] = df2.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)

In [18]:
df2.dtypes

DATE             object
ID               object
STOCK            object
SECTOR           object
TYPE             object
CONVICTIONS     float64
Time             object
dtype: object

In [19]:
df2['DATE'] = pd.to_datetime(
                          df2['DATE'],
                          format='%Y-%m-%d')

<ipython-input-19-3660468bee21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DATE'] = pd.to_datetime(


In [41]:
df2 = df2.sort_values(by="DATE")

## Adding returns

In [22]:
list_of_companies = df2["STOCK"].unique()

In [23]:
print(list_of_companies)

['SU' 'GGG' 'WGR' ... 'GBX' 'OVTI' 'SNCAF']


In [94]:
start_date = df2.DATE.min()
end_date = df2.DATE.max()

In [53]:
start_date = start_date + pd.DateOffset(months=6)
print(start_date)

2004-08-11 00:00:00


In [95]:
import yfinance as yf

#define the ticker symbol
tickerSymbol = 'SU'

#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker
tickerDf = tickerData.history(period='1m', start=start_date, end=end_date)

#see your data
tickerDf["STOCK"] = tickerSymbol
tickerDf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,STOCK
Date,,,,,,,,
2004-02-10,8.120850,8.370673,8.108200,8.332726,1641400,0.0,0,SU
2004-02-11,8.411778,8.452888,8.304259,8.402291,1280800,0.0,0,SU
2004-02-12,8.383322,8.383322,8.187258,8.196745,950400,0.0,0,SU
2004-02-13,8.301103,8.405460,8.073416,8.114526,1313400,0.0,0,SU
2004-02-17,8.285291,8.291615,8.149311,8.212557,1207000,0.0,0,SU
...,...,...,...,...,...,...,...,...
2004-07-07,7.921157,8.247901,7.898952,8.181284,2736200,0.0,0,SU
2004-07-08,8.216181,8.225698,8.108323,8.209836,1169400,0.0,0,SU
2004-07-09,8.216181,8.397000,8.184458,8.381139,1180000,0.0,0,SU


In [61]:
new_df = pd.merge(df2, tickerDf,  how='left', left_on=['STOCK','DATE'], right_on = ['STOCK','Date'])

In [136]:
def calculate_returns(start_date, df):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MRerturn','12MReturn'])
    
    date1m = start_date + pd.DateOffset(months=1)
    date3m = start_date + pd.DateOffset(months=3)
    date6m = start_date + pd.DateOffset(months=6)
    date12m = start_date +pd.DateOffset(months=12)

    current = df.loc[start_date]
    data1m = df.loc[date1m]
    data3m = df.loc[date3m]
    #data6m = df.loc[date6m]
    #data12m = df.loc[date12m]

    current = float(current['Close'])
    data1m = float(data1m['Close'])
    data3m = float(data3m['Close'])

    return1m = (data1m - current)/current
    return3m = (data3m - current)/current


    returns = pd.DataFrame([[start_date,"STOCK",return1m, return3m]], columns=['DATE','STOCK','1MReturn',"3MReturn"])

 

    return returns

In [137]:
test = calculate_returns(start_date, tickerDf)

print(test)

        DATE  STOCK  1MReturn  3MReturn
0 2004-02-11  STOCK  0.017189 -0.083814


In [106]:
tempdate = start_date +pd.DateOffset(months=1)
tickerDf.loc[start_date]

print(tempdate)
print(start_date)


tickerDf.loc[tempdate]

2004-03-11 00:00:00
2004-02-11 00:00:00


Open            8.51505
High            8.64488
Low             8.37888
Close           8.54672
Volume          1275000
Dividends             0
Stock Splits          0
STOCK                SU
Name: 2004-03-11 00:00:00, dtype: object